# Preprocess Natural Disasters

http://www.unitedstateszipcodes.org/zip-code-database/

- Download free
- Download csv

## Reading in the data

In [1]:
import pandas as pd
import numpy as np

disasters = pd.read_csv('DisasterDeclarationsSummaries.csv', usecols=[5,7,8,9,10,11,13,14], dtype=object,
                       header=0, names=['state_abbr', 'declaration_type', 'disaster_type', 
                                        'disaster_title', 'date_started', 'date_ended', 
                                        'county', 'fip_code'])
print disasters.info()
disasters.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46304 entries, 0 to 46303
Data columns (total 8 columns):
state_abbr          46304 non-null object
declaration_type    46304 non-null object
disaster_type       46304 non-null object
disaster_title      46304 non-null object
date_started        46304 non-null object
date_ended          45932 non-null object
county              46098 non-null object
fip_code            46087 non-null object
dtypes: object(8)
memory usage: 2.8+ MB
None


,state_abbr,declaration_type,disaster_type,disaster_title,date_started,date_ended,county,fip_code
0,LA,DR,Flood,FLOOD,1953-05-29T00:00:00 -04:00,1953-05-29T00:00:00 -04:00,NaN,NaN
1,GA,DR,Tornado,TORNADO,1953-05-02T00:00:00 -04:00,1953-05-02T00:00:00 -04:00,NaN,NaN
2,MI,DR,Tornado,TORNADO,1953-06-02T00:00:00 -04:00,1953-06-02T00:00:00 -04:00,NaN,NaN
3,CA,DR,Flood,FLOOD & EROSION,1954-02-05T00:00:00 -05:00,1954-02-05T00:00:00 -05:00,NaN,NaN
4,CT,DR,Hurricane,HURRICANES,1954-09-17T00:00:00 -04:00,1954-09-17T00:00:00 -04:00,NaN,NaN


In [2]:
# used both these sites to determine what type of disaster we're dealing with
# https://www.fema.gov/pdf/library/2300_15ch7.pdf
# https://www.fema.gov/he/disasters

disasters['declaration_type'] = np.where(disasters.declaration_type == 'DR', 'Major Disaster Declaration',
                                        np.where(disasters.declaration_type == 'EM', 'Emergency Declaration',
                                                np.where(disasters.declaration_type == 'FM', 'Fire Management Assistance',
                                                        np.where(disasters.declaration_type == 'FS', 'Fire Supression Authorization',
                                                                None))))

## Updating the dates

In [3]:
# get dates into a datetime
disasters['date_started'] = disasters['date_started'].str.replace('T.*', '')
disasters['date_ended'] = disasters['date_ended'].str.replace('T.*', '')

# get the difference in days
disasters['days_lasted'] = pd.to_datetime(disasters['date_ended']) - pd.to_datetime(disasters['date_started'])

# http://pandas.pydata.org/pandas-docs/stable/timedeltas.html
# change to int
disasters['days_lasted'] = disasters['days_lasted'].dt.days

disasters.head()

,state_abbr,declaration_type,disaster_type,disaster_title,date_started,date_ended,county,fip_code,days_lasted
0,LA,Major Disaster Declaration,Flood,FLOOD,1953-05-29,1953-05-29,NaN,NaN,0.0
1,GA,Major Disaster Declaration,Tornado,TORNADO,1953-05-02,1953-05-02,NaN,NaN,0.0
2,MI,Major Disaster Declaration,Tornado,TORNADO,1953-06-02,1953-06-02,NaN,NaN,0.0
3,CA,Major Disaster Declaration,Flood,FLOOD & EROSION,1954-02-05,1954-02-05,NaN,NaN,0.0
4,CT,Major Disaster Declaration,Hurricane,HURRICANES,1954-09-17,1954-09-17,NaN,NaN,0.0


## Subsetting data

In [4]:
# check the unique values from incident type
disasters['disaster_type'].unique()

array(['Flood', 'Tornado', 'Hurricane', 'Fire', 'Other', 'Severe Storm(s)',
       'Earthquake', 'Typhoon', 'Drought', 'Volcano', 'Dam/Levee Break',
       'Toxic Substances', 'Snow', 'Severe Ice Storm', 'Freezing',
       'Coastal Storm', 'Fishing Losses', 'Mud/Landslide', 'Human Cause',
       'Terrorist', 'Chemical', 'Tsunami'], dtype=object)

In [5]:
# We want only specific natural disasters and up to 2013
dtype = ['Tornado', 'Hurricane', 'Fire', 'Earthquake', 
         'Typhoon', 'Volcano', 'Coastal Storm', 'Tsunami']

cond1 = disasters['disaster_type'].isin(dtype)
cond2 = disasters['county'].notnull()           # need a county to geocode

disasters = disasters[cond1 & cond2]

In [6]:
disasters.shape

(13682, 9)

### Check if any states not in the state table

In [7]:
# cond = disasters['state_abbr'].isin(state_table['abbreviation'])
# print disasters.loc[~cond].shape
# disasters.loc[~cond].head()

We can see some irrelevant cases such as American Samoa (AS) that we don't need

## Cleaning the FIP codes

Here I grabbed FIP codes from 2010 from the census bureau 

https://www.census.gov/geo/reference/codes/cou.html

Download -> United States -> Go (opens .txt file in broswer and was downloaded)

In [8]:
fips = pd.read_csv('national_county.txt', header=0, dtype=object,
                   names=['state_abbr', 'state_fip', 'county_fip', 'county', 'fip_class_code'])
fips.head()

,state_abbr,state_fip,county_fip,county,fip_class_code
0,AL,01,003,Baldwin County,H1
1,AL,01,005,Barbour County,H1
2,AL,01,007,Bibb County,H1
3,AL,01,009,Blount County,H1
4,AL,01,011,Bullock County,H1


In [9]:
fips['county_fip'] = fips['state_abbr'] + '-' + fips['county_fip']
fips.head()

,state_abbr,state_fip,county_fip,county,fip_class_code
0,AL,01,AL-003,Baldwin County,H1
1,AL,01,AL-005,Barbour County,H1
2,AL,01,AL-007,Bibb County,H1
3,AL,01,AL-009,Blount County,H1
4,AL,01,AL-011,Bullock County,H1


Because the FIPs CAN repeat per state, then we can make a unique version by attaching the state abbreviation and the FIP code itself per county. We can also see that in the disasters dataset the FIP codes have 2 digits before the actual code which will need to be removed, this way when we try and compare plots of UFO sightings county names wont overlap and we can use FIP codes

In [10]:
disasters['fip_code'] = disasters['fip_code'].str.replace('^[0-9]{2}', '')

disasters['fip_code'] = disasters['state_abbr'] + '-' + disasters['fip_code']

## Cleaning and Merging data

In [11]:
state_table = pd.read_csv('state_table.csv')
stble_cols = ['abbreviation', 'name', 'census_region_name', 'census_division_name']

disasters = disasters.merge(state_table[stble_cols], how='inner', left_on='state_abbr', right_on='abbreviation')

del disasters['abbreviation']

# set the index to the start/end_date and get up to 2013
disasters = disasters.set_index(['date_started', 'date_ended'])
disasters = disasters.sort_index()
disasters = disasters.loc[:'2014']

# rename the columns added
disasters = disasters.rename(columns={'name':'state', 'census_region_name':'region_name', 'census_division_name':'division_name'})

# reorder DF 
reorder_cols = ['declaration_type', 'disaster_type','disaster_title', 'days_lasted', 'county', 'state', 
                'state_abbr','division_name','region_name', 'fip_code']

disasters = disasters[reorder_cols]

# have the title in title form
disasters['disaster_title'] = disasters['disaster_title'].str.title()

In [12]:
disasters.shape

(12484, 10)

In [13]:
# types of indexing
# disasters.loc['1965-04':'1965-05']
# disasters.loc[(slice(None), slice('1965-04', '1965-07')),:]

### Here we check to see which disasters FIP code arent in the .txt file

In [14]:
cond = disasters['fip_code'].isin(fips['county_fip'])
# cond2 = fips['county_fip'].isin(disasters['fip_code'])
print disasters.loc[~cond].shape
disasters.loc[~cond]

(253, 10)


,,declaration_type,disaster_type,disaster_title,days_lasted,county,state,state_abbr,division_name,region_name,fip_code
date_started,date_ended,,,,,,,,,,
1965-09-10,1965-09-10,Major Disaster Declaration,Hurricane,Hurricane Betsy,0.0,Ward 9 (Police Jury Ward),Louisiana,LA,West South Central,South,LA-440
1965-09-14,1965-09-14,Major Disaster Declaration,Hurricane,Hurricane Betsy,0.0,Dade (County),Florida,FL,South Atlantic,South,FL-025
1969-08-19,1969-08-19,Major Disaster Declaration,Hurricane,Hurricane Camille,0.0,Ward 9 (Police Jury Ward),Louisiana,LA,West South Central,South,LA-440
1974-09-23,1974-09-23,Major Disaster Declaration,Hurricane,Hurricane Carmen,0.0,Ward 9 (Police Jury Ward),Louisiana,LA,West South Central,South,LA-440
1975-12-18,1975-12-18,Major Disaster Declaration,Fire,Fire & Freezing,0.0,Tanana (ANV/ANVSA),Alaska,AK,Pacific,West,AK-165
1979-04-21,1979-04-21,Major Disaster Declaration,Tornado,"Tornadoes, Torrential Rain & Flooding",0.0,St. Louis,Missouri,MO,West North Central,Midwest,MO-000
1985-10-26,1985-11-08,Major Disaster Declaration,Hurricane,Hurricane Juan,13.0,Ward 9 (Police Jury Ward),Louisiana,LA,West South Central,South,LA-440
1987-11-15,1987-11-19,Major Disaster Declaration,Tornado,Tornadoes & Flooding,4.0,Ward 9 (Police Jury Ward),Louisiana,LA,West South Central,South,LA-440
1988-02-10,1988-02-10,Major Disaster Declaration,Fire,Fire,0.0,Tanana (ANV/ANVSA),Alaska,AK,Pacific,West,AK-165


In [15]:
# merging the fips dataset to this disasters to check if county names match up
fips_col = ['county', 'county_fip']

disasters = disasters.reset_index().merge(fips[fips_col], how='left', left_on='fip_code', right_on='county_fip').set_index(['date_started', 'date_ended'])

In [16]:
disasters.head()

declaration_type disaster_type  \
date_started date_ended                                             
1965-04-14   1965-04-14  Major Disaster Declaration       Tornado   
             1965-04-14  Major Disaster Declaration       Tornado   
             1965-04-14  Major Disaster Declaration       Tornado   
             1965-04-14  Major Disaster Declaration       Tornado   
             1965-04-14  Major Disaster Declaration       Tornado   

                                    disaster_title  days_lasted  \
date_started date_ended                                           
1965-04-14   1965-04-14  Tornadoes & Severe Storms          0.0   
             1965-04-14  Tornadoes & Severe Storms          0.0   
             1965-04-14  Tornadoes & Severe Storms          0.0   
             1965-04-14  Tornadoes & Severe Storms          0.0   
             1965-04-14  Tornadoes & Severe Storms          0.0   

                                 county_x    state state_abbr  \
date_started date_ended                                         
1965-04-14   1965-04-14   Howard (County)  Indiana         IN   
             1965-04-14    Grant (County)  Indiana         IN   
             1965-04-14    Wells (County)  Indiana         IN   
             1965-04-14  Clinton (County)  Indiana         IN   
             1965-04-14   Starke (County)  Indiana         IN   

                              division_name region_name fip_code  \
date_started date_ended                                            
1965-04-14   1965-04-14  East North Central     Midwest   IN-067   
             1965-04-14  East North Central     Midwest   IN-053   
             1965-04-14  East North Central     Midwest   IN-179   
             1965-04-14  East North Central     Midwest   IN-023   
             1965-04-14  East North Central     Midwest   IN-149   

                               county_y county_fip  
date_started date_ended                             
1965-04-14   1965-04-14   Howard County     IN-067  
             1965-04-14    Grant County     IN-053  
             1965-04-14    Wells County     IN-179  
             1965-04-14  Clinton County     IN-023  
             1965-04-14   Starke County     IN-149

In [17]:
disasters.shape

(12484, 12)

In [18]:
# remove the parenthesis 
disasters['county_x'] = disasters['county_x'].str.replace('(\()|(\))', '')

In [19]:
disasters.head()

declaration_type disaster_type  \
date_started date_ended                                             
1965-04-14   1965-04-14  Major Disaster Declaration       Tornado   
             1965-04-14  Major Disaster Declaration       Tornado   
             1965-04-14  Major Disaster Declaration       Tornado   
             1965-04-14  Major Disaster Declaration       Tornado   
             1965-04-14  Major Disaster Declaration       Tornado   

                                    disaster_title  days_lasted  \
date_started date_ended                                           
1965-04-14   1965-04-14  Tornadoes & Severe Storms          0.0   
             1965-04-14  Tornadoes & Severe Storms          0.0   
             1965-04-14  Tornadoes & Severe Storms          0.0   
             1965-04-14  Tornadoes & Severe Storms          0.0   
             1965-04-14  Tornadoes & Severe Storms          0.0   

                               county_x    state state_abbr  \
date_started date_ended                                       
1965-04-14   1965-04-14   Howard County  Indiana         IN   
             1965-04-14    Grant County  Indiana         IN   
             1965-04-14    Wells County  Indiana         IN   
             1965-04-14  Clinton County  Indiana         IN   
             1965-04-14   Starke County  Indiana         IN   

                              division_name region_name fip_code  \
date_started date_ended                                            
1965-04-14   1965-04-14  East North Central     Midwest   IN-067   
             1965-04-14  East North Central     Midwest   IN-053   
             1965-04-14  East North Central     Midwest   IN-179   
             1965-04-14  East North Central     Midwest   IN-023   
             1965-04-14  East North Central     Midwest   IN-149   

                               county_y county_fip  
date_started date_ended                             
1965-04-14   1965-04-14   Howard County     IN-067  
             1965-04-14    Grant County     IN-053  
             1965-04-14    Wells County     IN-179  
             1965-04-14  Clinton County     IN-023  
             1965-04-14   Starke County     IN-149

In [20]:
# county names are different but same place
disasters[disasters.county_x != disasters.county_y].head()

,,declaration_type,disaster_type,disaster_title,days_lasted,county_x,state,state_abbr,division_name,region_name,fip_code,county_y,county_fip
date_started,date_ended,,,,,,,,,,,,
1965-09-10,1965-09-10,Major Disaster Declaration,Hurricane,Hurricane Betsy,0.0,Ward 9 Police Jury Ward,Louisiana,LA,West South Central,South,LA-440,NaN,NaN
1965-09-14,1965-09-14,Major Disaster Declaration,Hurricane,Hurricane Betsy,0.0,Dade County,Florida,FL,South Atlantic,South,FL-025,NaN,NaN
1967-04-25,1967-04-25,Major Disaster Declaration,Tornado,Tornadoes,0.0,La Salle County,Illinois,IL,East North Central,Midwest,IL-099,LaSalle County,IL-099
1969-08-19,1969-08-19,Major Disaster Declaration,Hurricane,Hurricane Camille,0.0,Ward 9 Police Jury Ward,Louisiana,LA,West South Central,South,LA-440,NaN,NaN
1972-03-18,1972-03-18,Major Disaster Declaration,Coastal Storm,Coastal Storms,0.0,"Rockingham Countyin PMSA 4160,4760,6450",New Hampshire,NH,New England,Northeast,NH-015,Rockingham County,NH-015


In [21]:
disasters['county'] = np.where((disasters.county_y.isnull() & disasters.county_x.notnull()), 
                               disasters.county_x, 
                               disasters.county_y)

In [22]:
disasters.head()

declaration_type disaster_type  \
date_started date_ended                                             
1965-04-14   1965-04-14  Major Disaster Declaration       Tornado   
             1965-04-14  Major Disaster Declaration       Tornado   
             1965-04-14  Major Disaster Declaration       Tornado   
             1965-04-14  Major Disaster Declaration       Tornado   
             1965-04-14  Major Disaster Declaration       Tornado   

                                    disaster_title  days_lasted  \
date_started date_ended                                           
1965-04-14   1965-04-14  Tornadoes & Severe Storms          0.0   
             1965-04-14  Tornadoes & Severe Storms          0.0   
             1965-04-14  Tornadoes & Severe Storms          0.0   
             1965-04-14  Tornadoes & Severe Storms          0.0   
             1965-04-14  Tornadoes & Severe Storms          0.0   

                               county_x    state state_abbr  \
date_started date_ended                                       
1965-04-14   1965-04-14   Howard County  Indiana         IN   
             1965-04-14    Grant County  Indiana         IN   
             1965-04-14    Wells County  Indiana         IN   
             1965-04-14  Clinton County  Indiana         IN   
             1965-04-14   Starke County  Indiana         IN   

                              division_name region_name fip_code  \
date_started date_ended                                            
1965-04-14   1965-04-14  East North Central     Midwest   IN-067   
             1965-04-14  East North Central     Midwest   IN-053   
             1965-04-14  East North Central     Midwest   IN-179   
             1965-04-14  East North Central     Midwest   IN-023   
             1965-04-14  East North Central     Midwest   IN-149   

                               county_y county_fip          county  
date_started date_ended                                             
1965-04-14   1965-04-14   Howard County     IN-067   Howard County  
             1965-04-14    Grant County     IN-053    Grant County  
             1965-04-14    Wells County     IN-179    Wells County  
             1965-04-14  Clinton County     IN-023  Clinton County  
             1965-04-14   Starke County     IN-149   Starke County

In [23]:
del disasters['county_x']
del disasters['county_y']
del disasters['fip_code']

reorder_cols = ['declaration_type', 'disaster_type','disaster_title', 'days_lasted', 'county', 'county_fip', 'state', 
                'state_abbr','division_name','region_name']

disasters = disasters[reorder_cols]

In [24]:
disasters.shape

(12484, 10)

In [25]:
disasters.head()

declaration_type disaster_type  \
date_started date_ended                                             
1965-04-14   1965-04-14  Major Disaster Declaration       Tornado   
             1965-04-14  Major Disaster Declaration       Tornado   
             1965-04-14  Major Disaster Declaration       Tornado   
             1965-04-14  Major Disaster Declaration       Tornado   
             1965-04-14  Major Disaster Declaration       Tornado   

                                    disaster_title  days_lasted  \
date_started date_ended                                           
1965-04-14   1965-04-14  Tornadoes & Severe Storms          0.0   
             1965-04-14  Tornadoes & Severe Storms          0.0   
             1965-04-14  Tornadoes & Severe Storms          0.0   
             1965-04-14  Tornadoes & Severe Storms          0.0   
             1965-04-14  Tornadoes & Severe Storms          0.0   

                                 county county_fip    state state_abbr  \
date_started date_ended                                                  
1965-04-14   1965-04-14   Howard County     IN-067  Indiana         IN   
             1965-04-14    Grant County     IN-053  Indiana         IN   
             1965-04-14    Wells County     IN-179  Indiana         IN   
             1965-04-14  Clinton County     IN-023  Indiana         IN   
             1965-04-14   Starke County     IN-149  Indiana         IN   

                              division_name region_name  
date_started date_ended                                  
1965-04-14   1965-04-14  East North Central     Midwest  
             1965-04-14  East North Central     Midwest  
             1965-04-14  East North Central     Midwest  
             1965-04-14  East North Central     Midwest  
             1965-04-14  East North Central     Midwest

In [26]:
# check to see which county_fip is null
dis_na = disasters[disasters.county_fip.isnull()]

In [27]:
dis_na.shape

(253, 10)

In [28]:
# from looking we can see that theres a lot of missing in Virginia 
# from looking also at the county file, there should be a city so 
# ill add city to the name of the county to match before merging agian
# we'll update the county_fip_y for Dade County to be 

dis_na['county'] = np.where(dis_na.state_abbr == 'VA', dis_na.county + ' city', 
                            np.where(dis_na.county == 'Baltimore', dis_na.county + ' County', 
                                     np.where((dis_na.county == 'Dade County') & (dis_na.state_abbr == 'FL'), 'Miami-' + dis_na.county,
                                     dis_na.county)))

/anaconda/envs/py27/lib/python2.7/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [29]:
dis_na

,,declaration_type,disaster_type,disaster_title,days_lasted,county,county_fip,state,state_abbr,division_name,region_name
date_started,date_ended,,,,,,,,,,
1965-09-10,1965-09-10,Major Disaster Declaration,Hurricane,Hurricane Betsy,0.0,Ward 9 Police Jury Ward,NaN,Louisiana,LA,West South Central,South
1965-09-14,1965-09-14,Major Disaster Declaration,Hurricane,Hurricane Betsy,0.0,Miami-Dade County,NaN,Florida,FL,South Atlantic,South
1969-08-19,1969-08-19,Major Disaster Declaration,Hurricane,Hurricane Camille,0.0,Ward 9 Police Jury Ward,NaN,Louisiana,LA,West South Central,South
1974-09-23,1974-09-23,Major Disaster Declaration,Hurricane,Hurricane Carmen,0.0,Ward 9 Police Jury Ward,NaN,Louisiana,LA,West South Central,South
1975-12-18,1975-12-18,Major Disaster Declaration,Fire,Fire & Freezing,0.0,Tanana ANV/ANVSA,NaN,Alaska,AK,Pacific,West
1979-04-21,1979-04-21,Major Disaster Declaration,Tornado,"Tornadoes, Torrential Rain & Flooding",0.0,St. Louis,NaN,Missouri,MO,West North Central,Midwest
1985-10-26,1985-11-08,Major Disaster Declaration,Hurricane,Hurricane Juan,13.0,Ward 9 Police Jury Ward,NaN,Louisiana,LA,West South Central,South
1987-11-15,1987-11-19,Major Disaster Declaration,Tornado,Tornadoes & Flooding,4.0,Ward 9 Police Jury Ward,NaN,Louisiana,LA,West South Central,South
1988-02-10,1988-02-10,Major Disaster Declaration,Fire,Fire,0.0,Tanana ANV/ANVSA,NaN,Alaska,AK,Pacific,West


In [30]:
# remerge agian the data by name and the remove dulicates
dis_na = dis_na.reset_index().merge(fips, how='left', left_on=['state_abbr', 'county'], right_on=['state_abbr', 'county']).set_index(['date_started', 'date_ended'])


In [31]:
dis_na.shape

(253, 13)

In [32]:
dis_na.head()

,,declaration_type,disaster_type,disaster_title,days_lasted,county,county_fip_x,state,state_abbr,division_name,region_name,state_fip,county_fip_y,fip_class_code
date_started,date_ended,,,,,,,,,,,,,
1965-09-10,1965-09-10,Major Disaster Declaration,Hurricane,Hurricane Betsy,0.0,Ward 9 Police Jury Ward,NaN,Louisiana,LA,West South Central,South,NaN,NaN,NaN
1965-09-14,1965-09-14,Major Disaster Declaration,Hurricane,Hurricane Betsy,0.0,Miami-Dade County,NaN,Florida,FL,South Atlantic,South,12,FL-086,H1
1969-08-19,1969-08-19,Major Disaster Declaration,Hurricane,Hurricane Camille,0.0,Ward 9 Police Jury Ward,NaN,Louisiana,LA,West South Central,South,NaN,NaN,NaN
1974-09-23,1974-09-23,Major Disaster Declaration,Hurricane,Hurricane Carmen,0.0,Ward 9 Police Jury Ward,NaN,Louisiana,LA,West South Central,South,NaN,NaN,NaN
1975-12-18,1975-12-18,Major Disaster Declaration,Fire,Fire & Freezing,0.0,Tanana ANV/ANVSA,NaN,Alaska,AK,Pacific,West,NaN,NaN,NaN


In [33]:
# drop/reorder the columns
del dis_na['county_fip_x']
del dis_na['state_fip']
del dis_na['fip_class_code']

reorder_cols = ['declaration_type', 'disaster_type','disaster_title', 'days_lasted', 'county', 'county_fip_y', 'state', 
                'state_abbr','division_name','region_name']

dis_na = dis_na[reorder_cols]

dis_na = dis_na.rename(columns={'county_fip_y':'county_fip'})

In [34]:
# now we can remove the nulls from disasters
# rename it back to disasters and then combine
# dis_na back to disasters so theres no duplicate info
# then reorder the index again
disasters = disasters[disasters.county_fip.notnull()]

In [35]:
disasters.shape

(12231, 10)

In [36]:
disasters = pd.concat([disasters, dis_na])

In [37]:
disasters = disasters.sort_index()

In [38]:
disasters.shape

(12484, 10)

In [39]:
disasters.to_csv('disasters.csv')

# Using geocoder to get Latitude and Longitude for counties

In [25]:
import pandas as pd
import geocoder
import pickle

disasters = pd.read_csv('disasters.csv', parse_dates=True, index_col=['date_started', 'date_ended'])
D = pickle.load( open("dis_latlong.p", "rb") )
                      
county_state = disasters[['county', 'state']].values.tolist()

In [39]:
# for finding out how far we're in the file
# D = {}
row_counter = 0

# loop through the county_state list
for d in county_state:
    key = ', '.join(d)
    row_counter += 1

    if key not in D:
        g = geocoder.google(key)
        # stop trying to query
        if g.error == 'OVER_QUERY_LIMIT':
            raise Exception('Query Limit Exceeded')
            
        D[key] = {'latitude':g.lat, 'longitude':g.lng}
    else:
        pass

Exception: Query Limit Exceeded

In [40]:
row_counter

6603

In [41]:
# check how big our data structure is now
len(D)

2501

In [42]:
# save the data strucutre  if signifacant increase
pickle.dump( D, open("dis_latlong.p", "wb") )

# Combine data structure with dataset

In [40]:
import pandas as pd
import geocoder
import pickle

disasters = pd.read_csv('disasters.csv', parse_dates=True, index_col=['date_started', 'date_ended'])
D = pickle.load( open("dis_latlong.p", "rb") )

In [42]:
D

{'Imperial County, California': {'latitude': 33.0113694,
  'longitude': -115.4733554},
 'Henderson County, North Carolina': {'latitude': 35.3725873,
  'longitude': -82.4319405},
 'Saguache County, Colorado': {'latitude': 38.0019623,
  'longitude': -106.3463791},
 'Monmouth County, New Jersey': {'latitude': 40.2589455,
  'longitude': -74.12399599999999},
 'Ottawa County, Oklahoma': {'latitude': 36.7710787,
  'longitude': -94.81059549999999},
 'Grayson County, Kentucky': {'latitude': 37.4554962,
  'longitude': -86.3782198},
 'Conejos County, Colorado': {'latitude': 37.2689711,
  'longitude': -106.2522143},
 'Richmond County, New York': {'latitude': 40.5795317,
  'longitude': -74.1502007},
 'Douglas County, Wisconsin': {'latitude': 46.3490012,
  'longitude': -91.75768889999999},
 'Wilcox County, Alabama': {'latitude': 32.0105439, 'longitude': -87.3413599},
 'Stark County, Illinois': {'latitude': 41.0795661,
  'longitude': -89.81303559999999},
 'York County, Virginia': {'latitude': 37.2103

In [43]:
# make lat/long strings and a new temp column
county_state = disasters[['county','state']].astype(str).values.tolist()
disasters['county_state_key'] = [', '.join(x) for x in county_state]

# map the latlong_key to the dictionary L
disasters['county_state'] = disasters['county_state_key'].map(D)

In [46]:
disasters = pd.concat([disasters.drop(['county_state'], axis=1), disasters['county_state'].apply(pd.Series)], axis=1)

In [51]:
del disasters['county_state_key']

In [58]:
# get rid of any rows without a latitude or longitude
disasters = disasters[disasters.latitude.notnull()]

In [59]:
disasters.to_csv('disasters.csv')

In [60]:
disasters.shape

(12471, 12)